In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('updated_coswara.csv')

In [2]:
df

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc{1},mfcc{2},mfcc{3},mfcc{4},...,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,label
0,0.039102,0.378903,786.823461,966.699650,1387.984940,0.043870,-412.08945,126.752335,31.558170,18.483738,...,0.018672,0.021894,0.056015,0.212818,1.734878,0.097995,0.147546,0.206467,0.293984,1
1,0.051812,0.436672,2219.820298,1874.652272,4134.754998,0.209401,-398.93295,50.929253,-17.480385,4.325164,...,0.030713,0.037553,0.092139,0.283908,2.113383,0.137225,0.241707,0.116884,0.411675,0
2,0.010413,0.238371,2002.598308,2058.223470,4324.443295,0.146994,-599.82200,40.048190,6.373952,13.369130,...,0.018672,0.022602,0.056016,0.134015,1.358899,0.061172,0.091838,0.164672,0.183516,1
3,0.078437,0.242433,569.328347,506.956042,979.591497,0.035271,-423.74878,77.112580,-11.768955,6.080464,...,0.050445,0.070875,0.151334,0.181641,1.673087,0.101366,0.093390,0.116884,0.304098,0
4,0.072712,0.327475,1344.446613,1062.582139,2431.292693,0.089481,-327.21740,160.910540,-60.493977,-20.299130,...,0.001202,0.001537,0.003605,0.085836,0.803849,0.035992,0.054323,0.093975,0.107976,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24123,0.056854,0.467262,2845.574993,2044.704050,5269.369989,0.235494,-388.71793,32.990032,-8.879510,20.782476,...,0.023928,0.029485,0.071784,0.160208,1.659819,0.064901,0.137543,0.331425,0.194703,0
24124,0.050149,0.267105,1199.737564,1636.413852,2392.060470,0.059086,-433.10117,58.741127,37.350792,51.354850,...,0.002124,0.002396,0.006373,0.042722,0.419731,0.018084,0.027000,0.041864,0.054251,0
24125,0.000454,0.525523,4996.270042,2579.510698,8270.021928,0.418827,-745.49110,-27.318123,-5.540486,16.303144,...,0.016868,0.019114,0.050605,0.137923,1.255627,0.061327,0.081247,0.116884,0.183981,0
24126,0.031053,0.167581,1890.217497,2748.644250,5175.546000,0.033493,-499.73740,32.764600,46.792103,40.457275,...,0.001794,0.001854,0.005381,0.016609,0.134111,0.007814,0.009204,0.011837,0.023441,1


In [3]:
df = pd.DataFrame(df)

In [4]:

from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,BatchNormalization
from sklearn.model_selection import train_test_split



# Separate file names and labels from features
filenames = df.iloc[:, 0]
labels = df.iloc[:, -1]
features = df.iloc[:, 1:-1]




In [5]:
# Normalize features
scaler = StandardScaler()
features = scaler.fit_transform(features)


In [6]:
# Prepare the data
sequence_length = 10
n_features = features.shape[1]
sequences = []
targets = []


In [7]:
for i in range(sequence_length, len(df)-1):
    sequences.append(features[i-sequence_length:i])
    targets.append(labels[i-1])

X = np.array(sequences)
y = np.array(targets)

In [8]:
X = np.array(sequences)
y = np.array(targets)

In [9]:
# Slice filenames to match the number of samples in X and y
filenames = filenames[sequence_length:-1]

In [10]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test, filenames_train, filenames_test = train_test_split(X, y, filenames, test_size=0.2, random_state=42)


In [11]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(32, input_shape=(sequence_length, n_features)))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {accuracy}")

# Make predictions
predictions = model.predict(X_test)

# Save predictions and corresponding file names to a CSV file
results = pd.DataFrame({'filename': filenames_test, 'label': y_test, 'prediction': predictions.flatten()})
results.to_csv('predictions.csv', index=False)

Epoch 1/50
1206/1206 [==============================] - 24s 12ms/step - loss: 4.3785 - accuracy: 0.6059 - val_loss: 3.7703 - val_accuracy: 0.6482
Epoch 2/50
1206/1206 [==============================] - 12s 10ms/step - loss: 4.0964 - accuracy: 0.6307 - val_loss: 4.0847 - val_accuracy: 0.6374
Epoch 3/50
1206/1206 [==============================] - 13s 11ms/step - loss: 4.0462 - accuracy: 0.6433 - val_loss: 3.9460 - val_accuracy: 0.6590
Epoch 4/50
1206/1206 [==============================] - 13s 11ms/step - loss: 4.7012 - accuracy: 0.6162 - val_loss: 4.6903 - val_accuracy: 0.6097
Epoch 5/50
1206/1206 [==============================] - 11s 9ms/step - loss: 4.6003 - accuracy: 0.6123 - val_loss: 4.2072 - val_accuracy: 0.6370
Epoch 6/50
1206/1206 [==============================] - 11s 9ms/step - loss: 4.0359 - accuracy: 0.6443 - val_loss: 3.5769 - val_accuracy: 0.6702
Epoch 7/50
1206/1206 [==============================] - 12s 10ms/step - loss: 3.9057 - accuracy: 0.6460 - val_loss: 3.9549 - v

In [15]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
# Evaluate the model on test data
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn+fp)

# Print results
print(f"Test loss: {loss}")
print(f"Test accuracy: {accuracy}")
print(f"Test precision: {precision}")
print(f"Test recall: {recall}")
print(f"Test F1 score: {f1}")
print(f"Test ROC AUC: {auc}")
print(f"Test specificity: {specificity}")

151/151 [==============================] - 0s 3ms/step
Test loss: 0.5346227884292603
Test accuracy: 0.7742537260055542
Test precision: 0.753411306042885
Test recall: 0.7261625176139033
Test F1 score: 0.7395359961731642
Test ROC AUC: 0.7692037077865435
Test specificity: 0.8122448979591836


In [16]:

model.save('coswara_LSTM.h5')

# import pickle
# import joblib
# with open('coswara_LSTM.h5', 'wb') as f:
#     # Use pickle to dump the dictionary into the file
#     pickle.dump(model, f)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the model
model = Sequential([
    LSTM(units=64, return_sequences=True, input_shape=(sequence_length, n_features)),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=1)

# Make predictions on test set
y_pred = model.predict(X_test)

# Calculate the performance metrics
acc = accuracy_score(y_test, y_pred.round())
prec = precision_score(y_test, y_pred.round())
rec = recall_score(y_test, y_pred.round())
f1 = f1_score(y_test, y_pred.round())
auc = roc_auc_score(y_test, y_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("ROC AUC:", auc)


Epoch 1/50
603/603 [==============================] - 23s 28ms/step - loss: 0.6345 - accuracy: 0.6188 - val_loss: 0.4923 - val_accuracy: 0.7655
Epoch 2/50
603/603 [==============================] - 14s 23ms/step - loss: 0.4658 - accuracy: 0.7836 - val_loss: 0.4374 - val_accuracy: 0.8043
Epoch 3/50
603/603 [==============================] - 14s 23ms/step - loss: 0.4286 - accuracy: 0.8050 - val_loss: 0.4320 - val_accuracy: 0.8031
Epoch 4/50
603/603 [==============================] - 14s 24ms/step - loss: 0.4005 - accuracy: 0.8223 - val_loss: 0.4281 - val_accuracy: 0.8033
Epoch 5/50
603/603 [==============================] - 14s 23ms/step - loss: 0.3807 - accuracy: 0.8350 - val_loss: 0.4241 - val_accuracy: 0.8103
Epoch 6/50
603/603 [==============================] - 12s 20ms/step - loss: 0.3566 - accuracy: 0.8463 - val_loss: 0.4278 - val_accuracy: 0.8066
Epoch 7/50
603/603 [==============================] - 13s 21ms/step - loss: 0.3390 - accuracy: 0.8549 - val_loss: 0.4325 - val_accuracy:

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Define the model
model = Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(sequence_length, n_features)),
    Dropout(0.2),
    LSTM(units=128, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64, return_sequences=True),
    Dropout(0.2),
    LSTM(units=64),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32)

# Make predictions on test set
y_pred = model.predict(X_test)
y_pred_classes = [1 if x > 0.5 else 0 for x in y_pred]

# Compute evaluation metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)
roc_auc = roc_auc_score(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_classes).ravel()
specificity = tn / (tn+fp)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)
print("Specificity:", specificity)


Epoch 1/50
603/603 [==============================] - 34s 44ms/step - loss: 0.6893 - accuracy: 0.5475 - val_loss: 0.6860 - val_accuracy: 0.5587
Epoch 2/50
603/603 [==============================] - 26s 42ms/step - loss: 0.5924 - accuracy: 0.6643 - val_loss: 0.4790 - val_accuracy: 0.7751
Epoch 3/50
603/603 [==============================] - 26s 44ms/step - loss: 0.4504 - accuracy: 0.7964 - val_loss: 0.4469 - val_accuracy: 0.7933
Epoch 4/50
603/603 [==============================] - 25s 42ms/step - loss: 0.4145 - accuracy: 0.8129 - val_loss: 0.4363 - val_accuracy: 0.7989
Epoch 5/50
603/603 [==============================] - 20s 33ms/step - loss: 0.3787 - accuracy: 0.8334 - val_loss: 0.4524 - val_accuracy: 0.7981
Epoch 6/50
603/603 [==============================] - 24s 39ms/step - loss: 0.3451 - accuracy: 0.8528 - val_loss: 0.4451 - val_accuracy: 0.8016
Epoch 7/50
603/603 [==============================] - 21s 36ms/step - loss: 0.3070 - accuracy: 0.8731 - val_loss: 0.4792 - val_accuracy:

In [19]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Define the model architecture
def create_model(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    return model

# Define the evaluation metrics
def evaluate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn + fp)
    return accuracy, precision, recall, f1, roc_auc, specificity



# Define the cross-validation folds
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Iterate over the folds
fold = 5
for train_index, test_index in kf.split(X):
    # Split the data into train and test sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Create the model
    model = create_model(X_train.shape[1:])
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)
    
    # Make predictions on test set
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred)
    y_true = y_test
    
    # Evaluate the model on test set
    accuracy, precision, recall, f1, roc_auc, specificity = evaluate_metrics(y_true, y_pred)
    
    # Print the evaluation metrics
    print(f"Fold {fold}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 score: {f1}")
    print(f"ROC AUC score: {roc_auc}")
    print(f"Specificity: {specificity}")
    print("\n")
    
    # Increase the fold counter
    fold += 1


151/151 [==============================] - 2s 8ms/step
Fold 5
Accuracy: 0.7665837479270315
Precision: 0.7488833746898264
Recall: 0.7087834664161579
F1 score: 0.7282818532818535
ROC AUC score: 0.7605141821876708
Specificity: 0.8122448979591836


151/151 [==============================] - 2s 8ms/step
Fold 6
Accuracy: 0.7696932006633499
Precision: 0.7633018398806564
Recall: 0.7073732718894009
F1 score: 0.7342740971059556
ROC AUC score: 0.7640106751308345
Specificity: 0.8206480783722683


151/151 [==============================] - 3s 7ms/step
Fold 7
Accuracy: 0.7559610201119635
Precision: 0.726530612244898
Recall: 0.7362132352941176
F1 score: 0.7313398767404703
ROC AUC score: 0.7542040864796996
Specificity: 0.7721949376652815


151/151 [==============================] - 3s 7ms/step
Fold 8
Accuracy: 0.772133526850508
Precision: 0.7525325615050651
Recall: 0.7269338303821062
F1 score: 0.7395117326380658
ROC AUC score: 0.7676507030132421
Specificity: 0.808367575644378


151/151 [==============